# Gradient Boosting - Lab

## Introduction

In this lab, we'll learn how to use both Adaboost and Gradient Boosting classifiers from scikit-learn!

## Objectives

You will be able to:

- Use AdaBoost to make predictions on a dataset 
- Use Gradient Boosting to make predictions on a dataset 

## Getting Started

In this lab, we'll learn how to use boosting algorithms to make classifications on the [Pima Indians Dataset](http://ftp.ics.uci.edu/pub/machine-learning-databases/pima-indians-diabetes/pima-indians-diabetes.names). You will find the data stored in the file `'pima-indians-diabetes.csv'`. Our goal is to use boosting algorithms to determine whether a person has diabetes. Let's get started!

We'll begin by importing everything we need for this lab. Run cell below:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report

Now, use Pandas to import the data stored in `'pima-indians-diabetes.csv'` and store it in a DataFrame. Print the first five rows to inspect the data we've imported and ensure everything loaded correctly. 

In [2]:
# Import the data
OG_df = pd.read_csv('pima-indians-diabetes.csv')

# Print the first five rows
OG_df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


## Cleaning, exploration, and preprocessing

The target we're trying to predict is the `'Outcome'` column. A `1` denotes a patient with diabetes. 

By now, you're quite familiar with exploring and preprocessing a dataset.  

In the following cells:

* Check for missing values and deal with them as you see fit (if any exist) 
* Count the number of patients with and without diabetes in this dataset 
* Store the target column in a separate variable and remove it from the dataset
* Split the dataset into training and test sets, with a `test_size` of 0.25 and a `random_state` of 42

In [3]:
# Check for missing values

# The rows with 0 blood pressure almost uniformly also have 0 for skin thickness and insulin. As there are only 35 of these
# entries, and since they contain no information in several columns, they will be deleted. Glucose and BMI only represent a
# a few more points, so they will be deleted as well.
filtered_df = OG_df[(OG_df['BloodPressure'] > 0) & (OG_df['Glucose'] > 0) & (OG_df['BMI'] > 0)]

# Glucose, BP=0 (35), SkinThickness=0 (227), =99(1), Insulin=0(374), BMI=0(11)

In [4]:
# Number of patients with and without diabetes
outcomes = filtered_df['Outcome'].value_counts()
print(f'Positive: {round(outcomes[1]/(outcomes[0]+outcomes[1])*100,2)}%')

Positive: 34.39%


In [6]:
from sklearn.impute import SimpleImputer

imp = SimpleImputer(strategy='median', missing_values=0)
imp_df = imp.fit_transform(filtered_df.drop(['Pregnancies','Outcome'],axis=1))

imp_df = pd.DataFrame(
    imp_df,
    columns=filtered_df.drop(['Pregnancies','Outcome'],axis=1).columns,
    index=filtered_df.index)

imp_df['Pregnancies'] = filtered_df['Pregnancies'].copy()
imp_df

,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Pregnancies
0,148.0,72.0,35.0,125.5,33.6,0.627,50.0,6
1,85.0,66.0,29.0,125.5,26.6,0.351,31.0,1
2,183.0,64.0,29.0,125.5,23.3,0.672,32.0,8
3,89.0,66.0,23.0,94.0,28.1,0.167,21.0,1
4,137.0,40.0,35.0,168.0,43.1,2.288,33.0,0
...,...,...,...,...,...,...,...,...
763,101.0,76.0,48.0,180.0,32.9,0.171,63.0,10
764,122.0,70.0,27.0,125.5,36.8,0.340,27.0,2
765,121.0,72.0,23.0,112.0,26.2,0.245,30.0,5
766,126.0,60.0,29.0,125.5,30.1,0.349,47.0,1


In [7]:
target = filtered_df['Outcome']
df = imp_df #filtered_df.drop('Outcome', axis=1)

In [8]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df,target,random_state=42)

## Train the models

Now that we've explored the dataset, we're ready to fit some models!

In the cell below:

* Instantiate an `AdaBoostClassifier` (set the `random_state` for 42)
* Instantiate a `GradientBoostingClassifer` (set the `random_state` for 42) 

In [9]:
# Instantiate an AdaBoostClassifier
adaboost_clf = AdaBoostClassifier(random_state=42)

# Instantiate an GradientBoostingClassifier
gbt_clf = GradientBoostingClassifier(random_state=42)

Now, fit the training data to both the classifiers: 

In [10]:
# Fit AdaBoostClassifier
adaboost_clf.fit(X_train, y_train);

In [11]:
# Fit GradientBoostingClassifier
gbt_clf.fit(X_train, y_train);

Now, let's use these models to predict labels on both the training and test sets: 

In [12]:
# AdaBoost model predictions
adaboost_train_preds = adaboost_clf.predict(X_train)
adaboost_test_preds = adaboost_clf.predict(X_test)

# GradientBoosting model predictions
gbt_clf_train_preds = gbt_clf.predict(X_train)
gbt_clf_test_preds = gbt_clf.predict(X_test)

Now, complete the following function and use it to calculate the accuracy and f1-score for each model: 

In [13]:
def display_acc_and_f1_score(true, preds, model_name):
    acc = accuracy_score(true,preds)
    f1 = f1_score(true,preds)
    print("Model: {}".format(model_name))
    print("Accuracy: {:.2%}".format(acc))
    print("F1-Score: {:.2%}".format(f1))
    
print("Training Metrics\n")
display_acc_and_f1_score(y_train, adaboost_train_preds, model_name='AdaBoost')
print("")
display_acc_and_f1_score(y_train, gbt_clf_train_preds, model_name='Gradient Boosted Trees')
print("")
print("\n\nTesting Metrics\n")
display_acc_and_f1_score(y_test, adaboost_test_preds, model_name='AdaBoost')
print("")
display_acc_and_f1_score(y_test, gbt_clf_test_preds, model_name='Gradient Boosted Trees')

zero_pred = [0] * len(y_test)
print("")
display_acc_and_f1_score(y_test, zero_pred, model_name='All zeroes')


Training Metrics

Model: AdaBoost
Accuracy: 83.79%
F1-Score: 76.72%

Model: Gradient Boosted Trees
Accuracy: 93.74%
F1-Score: 90.96%



Testing Metrics

Model: AdaBoost
Accuracy: 79.56%
F1-Score: 68.38%

Model: Gradient Boosted Trees
Accuracy: 76.24%
F1-Score: 61.26%

Model: All zeroes
Accuracy: 69.61%
F1-Score: 0.00%


Let's go one step further and create a confusion matrix and classification report for each. Do so in the cell below: 

In [14]:
adaboost_confusion_matrix = confusion_matrix(y_test,adaboost_test_preds)
adaboost_confusion_matrix

array([[104,  22],
       [ 15,  40]], dtype=int64)

In [15]:
gbt_confusion_matrix = confusion_matrix(y_test,gbt_clf_test_preds)
gbt_confusion_matrix

array([[104,  22],
       [ 21,  34]], dtype=int64)

In [16]:
adaboost_classification_report = classification_report(y_test,adaboost_test_preds)
print(adaboost_classification_report)

              precision    recall  f1-score   support

           0       0.87      0.83      0.85       126
           1       0.65      0.73      0.68        55

    accuracy                           0.80       181
   macro avg       0.76      0.78      0.77       181
weighted avg       0.80      0.80      0.80       181



In [17]:
gbt_classification_report = classification_report(y_test,gbt_clf_test_preds)
print(gbt_classification_report)

              precision    recall  f1-score   support

           0       0.83      0.83      0.83       126
           1       0.61      0.62      0.61        55

    accuracy                           0.76       181
   macro avg       0.72      0.72      0.72       181
weighted avg       0.76      0.76      0.76       181



**_Question:_** How did the models perform? Interpret the evaluation metrics above to answer this question.

Write your answer below this line:
_______________________________________________________________________________________________________________________________
These both did better than the all zeroes model. Between the two, adaboost performed slightly better.
 
 
As a final performance check, let's calculate the 5-fold cross-validated score for each model! 

Recall that to compute the cross-validation score, we need to pass in:

* A classifier
* All training data
* All labels
* The number of folds we want in our cross-validation score  

Since we're computing cross-validation score, we'll want to pass in the entire dataset, as well as all of the labels. 

In the cells below, compute the mean cross validation score for each model. 

In [18]:
print('Mean Adaboost Cross-Val Score (k=5):')
print(cross_val_score(adaboost_clf, X_train,y_train,cv=5).mean())
# Expected Output: 0.7631270690094218

Mean Adaboost Cross-Val Score (k=5):
0.7274549779136935


In [19]:
print('Mean GBT Cross-Val Score (k=5):')
print(cross_val_score(gbt_clf, X_train,y_train,cv=5).mean())
# Expected Output: 0.7591715474068416

Mean GBT Cross-Val Score (k=5):
0.736561331974176


These models didn't do poorly, but we could probably do a bit better by tuning some of the important parameters such as the **_Learning Rate_**. 

## Summary

In this lab, we learned how to use scikit-learn's implementations of popular boosting algorithms such as AdaBoost and Gradient Boosted Trees to make classification predictions on a real-world dataset!